In [ ]:
import tensorflow
def decode_urlsafe_b64encode_image(image_64_encode):
    # decode string to binary
    # image_64_decoded = tensorflow.io.decode_base64(image_64_encode)
    # decode binary to image
    image_decoded = tensorflow.io.decode_jpeg(image_64_encode, channels=3)
    # resize image
    resized_image = tensorflow.image.resize_images(image_decoded, (224, 224), method=tensorflow.image.ResizeMethod.BILINEAR, align_corners=False)
    # cast tensor to uint8 for encoding
    resized_image = tensorflow.cast(resized_image, tensorflow.uint8)
    # normalizing scale
    scale = tensorflow.constant(255, dtype=tensorflow.uint8)
    # normalize image by division of 255.
    resized_image = tensorflow.math.divide(resized_image, scale)
    return resized_image

In [2]:
from tensorflow.keras.models import load_model

In [3]:
model = load_model('food-mobilenet-refine-05-0.967.hdf5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [5]:
#from tensorflow import keras
#keras.experimental.export_saved_model(model, 'originalmodel')

In [6]:
model.save_weights('food-mobilenet-refine-05-0.967.h5')

In [7]:
string_input_layer = tensorflow.keras.layers.Input(shape = (1,),dtype="string")

In [8]:
string_output_layer = tensorflow.keras.layers.Lambda(lambda img : tensorflow.map_fn(lambda im : decode_urlsafe_b64encode_image(im[0]), img, dtype="float32"))(string_input_layer)

In [9]:
string_output_layer

<tf.Tensor 'lambda/map/TensorArrayStack/TensorArrayGatherV3:0' shape=(?, 224, 224, 3) dtype=float32>

In [10]:
new_outputs = model(string_output_layer)

In [ ]:
'''
def c(x):
    return x
that_output_layer = tensorflow.keras.layers.Lambda(c)(new_outputs)
'''

In [11]:
from tensorflow.keras.models import Model
newModel = Model(string_input_layer, new_outputs)

In [12]:
#from tensorflow.keras.models import Model
#newModel = Model(string_input_layer, that_output_layer)

In [13]:
newModel.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
lambda (Lambda)              (None, 224, 224, 3)       0         
_________________________________________________________________
model_1 (Model)              (None, 2)                 3571778   
Total params: 3,571,778
Trainable params: 3,377,282
Non-trainable params: 194,496
_________________________________________________________________


In [14]:
newModel.predict

<bound method Model.predict of <tensorflow.python.keras.engine.training.Model object at 0x142ccf940>>

In [15]:
#from tensorflow import keras
#keras.experimental.export_saved_model(newModel, 'updatedmodel')

In [16]:
newModel.save('food-mobilenet-refine-05-0.967_add_layer_remove_base64.h5')

In [17]:
newModel.save_weights('food-mobilenet-refine-05-0.967_add_layer_remove_base64.h5')

In [18]:
newModel.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
lambda (Lambda)              (None, 224, 224, 3)       0         
_________________________________________________________________
model_1 (Model)              (None, 2)                 3571778   
Total params: 3,571,778
Trainable params: 3,377,282
Non-trainable params: 194,496
_________________________________________________________________


In [19]:
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

In [20]:
from tensorflow.keras import backend as K

In [21]:
!rm -rf food-mobilenet-refine-05-0.967_add_layer_remove_base64

In [22]:
builder = saved_model_builder.SavedModelBuilder("food-mobilenet-refine-05-0.967_add_layer_remove_base64")

In [23]:
signature = predict_signature_def(inputs={"images": newModel.input},outputs={"scores": newModel.output})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


In [24]:
with K.get_session() as sess:
    # Save the meta graph and the variables
    builder.add_meta_graph_and_variables(sess=sess, tags=[tag_constants.SERVING],
                                     signature_def_map={"predict": signature})

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.


In [25]:
builder.save()

INFO:tensorflow:SavedModel written to: food-mobilenet-refine-05-0.967_add_layer_remove_base64/saved_model.pb


b'food-mobilenet-refine-05-0.967_add_layer_remove_base64/saved_model.pb'

In [ ]:
# Load the model 

In [33]:
import base64

In [34]:
image = open('bridge.png', 'rb')

In [35]:
image

<_io.BufferedReader name='bridge.png'>

In [36]:
image_read = image.read()

In [37]:
image_64_encode = base64.urlsafe_b64encode(image_read)

In [ ]:
image_64_encode

In [38]:
sample = tensorflow.constant(image_64_encode)

In [39]:
sample.dtype

tf.string

In [40]:
with tensorflow.Session() as sess:
    #newModel.predict(sample.eval())
    print(sample.eval())

b'iVBORw0KGgoAAAANSUhEUgAABQAAAANYCAYAAACb6UBXAAAKqGlDQ1BJQ0MgUHJvZmlsZQAASImVlgdUU-kSgP970xst9Bp67wIBpIQeunSwEZIAoYSQEBRsqCyu4FoQEQF1RZciCjaKLDZEsaKggn1BFgV1XSyIipp3kUfcfe-8986bc-be78ydf_6Z_85_zgBAvs0SCDJgOQAy-TnCCH8vWlx8Ag33BEAAC2QAAGostkjACA8PRhjMvf8u7wcQb0RuWc7E-vfv_1XkOVwRGwAoHOEkjoidifAJRDvZAmEOACgBYtdfliOY4TKEFYVIggjXz3DKLHfOcNIs3_zmExXhjfDvAODJLJYwBQDSBGKn5bJTkDhkpFpgw-fw-Ah7IuzOTmVxEC5A2CIzM2uGDyFskvSXOCl_i5kkjclipUh5tpZvgvfhiQQZrLz_8zj-t2RmiOf20EeUnCoMiJjZb-bc0rOCpMxPCg2bYx5nNqcZThUHRM8xW-SdMMcclk-QdG1GaPAcJ_P8mNI4OcyoORZmRUjjc0W-kXPMEn7fS5wezZDuy2VKY-anRsXOcS4vJnSORemRQd99vKV2oThCmnOy0E9aY6boL3XxmFL_nNSoAGmNrO-5cUVx0hw4XB9fqZ0fLfUR5HhJ4wsywqX-3Ax_qV2UGyldm4M02_e14dLzSWMFhs8x4IEQwALsHO7ynJmEvbMEeUJeSmoOjYHcGC6NyWdbWdDsbGzpAMzcv9nfO3Hj272C1OS_24pUkHa9KpFIWr_bmEg_HOUhPWj63Wa8GADZ6wBcCmWLhbmzNvTMAwOIQBYoAjWgjfSPCbAEdsARuAJP4AsCQRiIAvFgCWCDVJAJhGAZWAnWgiJQAraCHaAS7AX7QT04DI6BNtAJzoGL4Cq4Ce6AB2AIjIIXYAK8B9MQBOEgCkSF1CAdyBAyh-wgOuQO-ULBUAQUDyVCKRAfEkMrofVQCVQKVUL7oAboKHQSOgddhvqge9

In [41]:
import numpy
kkk = numpy.array(image_64_encode)

In [42]:
newModel.predict(kkk)

ValueError: Error when checking input: expected input_1 to have 2 dimensions, but got array with shape ()

In [43]:
import numpy
a = numpy.array(image_64_encode)
a = a.reshape(1,1)

In [ ]:
model

In [44]:
newModel.predict(a)

RuntimeError: Attempted to use a closed Session.

In [ ]:
import numpy
a = numpy.array(image_64_encode)
a = a.reshape(1,1)

In [ ]:
newModel.predict(a)

In [26]:
# Other 

In [57]:
with tensorflow.Session() as sess:
    loaded = tensorflow.saved_model.load(sess = sess,  tags=[tag_constants.SERVING] , export_dir="food-mobilenet-refine-05-0.967_add_layer_remove_base64")
    type(loaded)
    #print(list(loaded.signatures.keys()))
    #print(list(loaded.signatures.keys()))  # ["serving_default"]

INFO:tensorflow:Restoring parameters from food-mobilenet-refine-05-0.967_add_layer_remove_base64/variables/variables


In [ ]:
! pwd

{'predict': inputs {
  key: "images"
  value {
    name: "input_1_1:0"
    dtype: DT_STRING
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 1
      }
    }
  }
}
outputs {
  key: "scores"
  value {
    name: "model_1/dense_2/Softmax:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 2
      }
    }
  }
}
method_name: "tensorflow/serving/predict"
}

In [ ]:
image_contents = tensorflow.read_file("bridge.png")
image = tensorflow.image.decode_jpeg(image_contents, channels=3)

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model = load_model('food-mobilenet-refine-05-0.967_add_layer_remove_base64.h5')

In [ ]:
a.dtype

In [56]:
tensorflow.__version__

'1.14.0'